# <b>ENGR 418 Project Stage 1</b>

<b>Group 7 <br>
Group Members:</b> <br>
Alif Aiman Ahmad Zukiman | 35338490 <br>
Anas Chellabi | 

_October 30, 2023_

## Problem Desription
We are to classify a Lego dataset into four different classes: 2x4 rectangles, 2x2 squares, 2x2 circles, and 2x1 rectangles. The classified Legos are then sorted according to their predicted classes. This will be done via the multi-class classification algorithm, as presented below.

### 1. Extracting Data

Import necessary libraries:

In [3]:
import os                                                       # for recognising path directories
import numpy as np  
from numpy import asarray
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression             # algorithm to be used
from sklearn.metrics import confusion_matrix, accuracy_score    # accuracy testing
from PIL import Image                                           # image processor

Assign arrays of training and testing by redirecting the arrays to the specified paths:

In [96]:
n_classes = 4   # no. of classes

# create empty lists for four classes
folder_training = []
folder_testing = [0 for i in range(n_classes)]
classes = [i for i in range(n_classes)]

# dataset file path for Alif
training_path = "E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\" 
testing_path = "E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\testing\\" 

# dataset file path for Anas
# training_path = "insert your path here" 
# testing_path = "insert your path here"

# detect how many files we have per class
# class 0:"2b1", class 1:"cir", class 2:"rec", class 3:"squ"
class_type = ["2b1", "cir", "rec", "squ"]
for i in range(len(class_type)):

    class_temp = [] # list to temporarily store class in consideration

    # run through the entire training dataset and detect if it is of the same type as
    # one we are considering. if true, then put into class_temp
    for j in os.listdir(training_path):
        if class_type[i] in j:
            class_temp.append(j)
    
    folder_training.append(class_temp)

for i in range(len(folder_training)):
    for j in range(len(folder_training[i])):
        folder_elem = folder_training[i][j]
        folder_training[i][j] = training_path + folder_elem

print(folder_training)


[['E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\2b1_1.png', 'E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\2b1_10.png', 'E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\2b1_11.png', 'E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\2b1_12.png', 'E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\2b1_13.png', 'E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\2b1_14.png', 'E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\2b1_15.png', 'E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\2b1_16.png', 'E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\2b1_17.png', 'E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\2b1_18.png', 'E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\2b1_19.png', 'E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\2b1_2.png', 'E:\

Define a function "get_data" to retrieve information about the chosen images:

In [99]:
def get_data(file_path, im_width, label, n_samples):

    # x,y coordinates for storing the image pixels
    x = np.empty((n_samples, im_width**2))
    y = np.empty((n_samples, 1))

    # extract image information into x,y
    for i in range(n_samples):
        im = Image.open(file_path).convert('L')      # open image and convert to grayscale
        im = im.resize((im_width,im_width))          # resize to pixel size desired
        im_array = asarray(im)
        x[i,:] = im_array.reshape(1,-1)
        y[i,0] = classes[label]

    return x,y

### 2. Parameter Initialization and Training Data Preparation

In [102]:
im_width = 64   # pixel size to be considered
P_per_class = [len(folder_training[i]) for i in range(n_classes)]

x_train = np.empty((sum(P_per_class), im_width**2))
y_train = np.empty((sum(P_per_class), 1))

for i in range(n_classes):
    for j in range(len(folder_training[i])):
        x_i, y_i = get_data(folder_training[i][j], im_width, i, P_per_class[i])
        x_train[i*P_per_class[i]:(i+1)*P_per_class[i],:] = x_i
        y_train[i*P_per_class[i]:(i+1)*P_per_class[i],:] = y_i

print(x_train.shape, y_train.shape)


    

(144, 4096) (144, 1)


### 3. Model Training

In [103]:
model = LogisticRegression()
model.fit(x_train,y_train)

c:\Users\Alif Aiman\anaconda3\Lib\site-packages\sklearn\utils\validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Alif Aiman\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()